In [48]:
from sympy import false, true
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pickle
import matplotlib.pyplot as plt
import optuna

c:\Users\veren\anaconda3\envs\ep_forecasting_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# Data Import 

# Load X_train
with open("data/X_train.pkl", "rb") as f:
    X_train = pickle.load(f)

# Load y_train
with open("data/y_train.pkl", "rb") as f:
    y_train = pickle.load(f)

# Load X_val
with open("data/X_val.pkl", "rb") as f:
    X_val = pickle.load(f)

# Load y_val
with open("data/y_val.pkl", "rb") as f:
    y_val = pickle.load(f)

# Load X_test
with open("data/X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

# Load y_test
with open("data/y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

# Load df_final_viz
with open("data/df_final_viz.pkl", "rb") as f:
    df_final_viz = pickle.load(f)

In [50]:
torch.set_num_threads(torch.get_num_threads())  # Nutzt standardmäßig alle verfügbaren Kerne
print(f"PyTorch nutzt {torch.get_num_threads()} Threads")


PyTorch nutzt 16 Threads


In [57]:
# In Torch-Tensoren umwandeln
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Datasets und DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last = True, num_workers = 8)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last = True, num_workers = 8)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last = True, num_workers = 8)

In [52]:
X_train.shape[0]

27024

In [58]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hp):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hp['rnn_units'], batch_first=True)
        self.dropout1 = nn.Dropout(hp['dropout_rate_rnn'])
        self.fc1 = nn.Linear(hp['rnn_units'], hp['dense_units'])
        self.dropout2 = nn.Dropout(hp['dropout_rate_dense'])
        self.fc2 = nn.Linear(hp['dense_units'], 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Nur der letzte Zeitschritt
        out = self.dropout1(out)
        out = self.fc1(out)
        out = self.dropout2(out)
        out = self.fc2(out)
        return out

In [ ]:
def objective(trial):
    hp = {
        'rnn_units': trial.suggest_int('rnn_units', 16, 128, step=16),
        'dropout_rate_rnn': trial.suggest_float('dropout_rate_rnn', 0.1, 0.5, step=0.1),
        'dense_units': trial.suggest_int('dense_units', 8, 64, step=8),
        'dropout_rate_dense': trial.suggest_float('dropout_rate_dense', 0.0, 0.4, step=0.1),
        'learning_rate': trial.suggest_categorical('learning_rate', [1e-2, 1e-3, 1e-4]),
        'weight_decay': trial.suggest_categorical('weight_decay', [1e-5, 1e-4, 1e-3]),
    }

    rnn_model = SimpleRNN(input_size = X_train.shape[2], hp= hp)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(rnn_model.parameters(), lr=hp['learning_rate'], weight_decay=hp['weight_decay'])

    num_epochs = 15
    patience = 7  # Stop, wenn val_loss sich 10 Epochen lang nicht verbessert
    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        rnn_model.train()
        train_loss = 0.0

        # Training Loop
        for X_batch, y_batch in train_loader:
            y_pred = rnn_model(X_batch)
            loss = criterion(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Validation Loop (nach jeder Epoche)
        rnn_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = rnn_model(X_batch)
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        # Early Stopping Check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0  # Reset Counter
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f"Early stopping nach {epoch+1} Epochen.")
                break

    return best_val_loss  # Val Loss zurückgeben für Optuna

# Hyperparameter tuning
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30, n_jobs = 6)

# Show Best Result
print("Best trial parameters:")
for key, value in study.best_trial.params.items():
    print(f"{key}: {value}")


[I 2025-03-31 09:31:39,517] A new study created in memory with name: no-name-97a31bca-2df9-42ba-9631-73e0aabf594f


Epoch [1/15], Train Loss: 0.0082, Val Loss: 0.0025
Epoch [1/15], Train Loss: 0.0301, Val Loss: 0.0102
Epoch [1/15], Train Loss: 0.0182, Val Loss: 0.0034
Epoch [1/15], Train Loss: 0.0056, Val Loss: 0.0019
Epoch [1/15], Train Loss: 0.0057, Val Loss: 0.0011
Epoch [1/15], Train Loss: 0.0133, Val Loss: 0.0028
Epoch [2/15], Train Loss: 0.0021, Val Loss: 0.0015
Epoch [2/15], Train Loss: 0.0134, Val Loss: 0.0043
Epoch [2/15], Train Loss: 0.0022, Val Loss: 0.0010
Epoch [2/15], Train Loss: 0.0073, Val Loss: 0.0019
Epoch [2/15], Train Loss: 0.0032, Val Loss: 0.0020
Epoch [3/15], Train Loss: 0.0016, Val Loss: 0.0010
Epoch [3/15], Train Loss: 0.0096, Val Loss: 0.0022
Epoch [2/15], Train Loss: 0.0041, Val Loss: 0.0021
Epoch [4/15], Train Loss: 0.0014, Val Loss: 0.0009
Epoch [4/15], Train Loss: 0.0080, Val Loss: 0.0022
Epoch [3/15], Train Loss: 0.0017, Val Loss: 0.0007
Epoch [3/15], Train Loss: 0.0061, Val Loss: 0.0020
Epoch [3/15], Train Loss: 0.0033, Val Loss: 0.0016
Epoch [3/15], Train Loss: 0.003

[I 2025-03-31 09:38:39,543] Trial 5 finished with value: 0.0008676108472789037 and parameters: {'rnn_units': 48, 'dropout_rate_rnn': 0.2, 'dense_units': 56, 'dropout_rate_dense': 0.0, 'learning_rate': 0.001, 'weight_decay': 0.001}. Best is trial 5 with value: 0.0008676108472789037.


Epoch [11/15], Train Loss: 0.0013, Val Loss: 0.0010
Early stopping nach 11 Epochen.
Epoch [7/15], Train Loss: 0.0163, Val Loss: 0.0090
Epoch [7/15], Train Loss: 0.0021, Val Loss: 0.0014
Epoch [8/15], Train Loss: 0.0014, Val Loss: 0.0007
Epoch [1/15], Train Loss: 0.0262, Val Loss: 0.0111
Epoch [11/15], Train Loss: 0.0031, Val Loss: 0.0013
Epoch [8/15], Train Loss: 0.0034, Val Loss: 0.0018


[I 2025-03-31 09:39:38,312] Trial 1 finished with value: 0.0010949653383689902 and parameters: {'rnn_units': 64, 'dropout_rate_rnn': 0.5, 'dense_units': 16, 'dropout_rate_dense': 0.4, 'learning_rate': 0.01, 'weight_decay': 0.0001}. Best is trial 5 with value: 0.0008676108472789037.


...

Epoch [9/15], Train Loss: 0.0039, Val Loss: 0.0023
Early stopping nach 9 Epochen.
Epoch [11/15], Train Loss: 0.0019, Val Loss: 0.0009
Epoch [1/15], Train Loss: 0.0054, Val Loss: 0.0015
Epoch [12/15], Train Loss: 0.0009, Val Loss: 0.0007
Epoch [2/15], Train Loss: 0.0050, Val Loss: 0.0025
Epoch [12/15], Train Loss: 0.0019, Val Loss: 0.0010
Epoch [1/15], Train Loss: 0.0056, Val Loss: 0.0011
Epoch [14/15], Train Loss: 0.0015, Val Loss: 0.0008
Epoch [13/15], Train Loss: 0.0019, Val Loss: 0.0010
Epoch [2/15], Train Loss: 0.0023, Val Loss: 0.0008
Epoch [13/15], Train Loss: 0.0009, Val Loss: 0.0007
Epoch [3/15], Train Loss: 0.0038, Val Loss: 0.0017
Epoch [2/15], Train Loss: 0.0023, Val Loss: 0.0009
Epoch [14/15], Train Loss: 0.0019, Val Loss: 0.0010


[I 2025-03-31 09:57:39,581] Trial 8 finished with value: 0.0006972745170294788 and parameters: {'rnn_units': 112, 'dropout_rate_rnn': 0.30000000000000004, 'dense_units': 8, 'dropout_rate_dense': 0.2, 'learning_rate': 0.001, 'weight_decay': 0.0001}. Best is trial 12 with value: 0.0006254629491107358.


Epoch [15/15], Train Loss: 0.0016, Val Loss: 0.0008
Epoch [3/15], Train Loss: 0.0015, Val Loss: 0.0007


[I 2025-03-31 09:57:54,395] Trial 13 finished with value: 0.00092358139215409 and parameters: {'rnn_units': 16, 'dropout_rate_rnn': 0.4, 'dense_units': 56, 'dropout_rate_dense': 0.0, 'learning_rate': 0.001, 'weight_decay': 0.001}. Best is trial 12 with value: 0.0006254629491107358.


Epoch [15/15], Train Loss: 0.0019, Val Loss: 0.0011
Epoch [14/15], Train Loss: 0.0009, Val Loss: 0.0007
Epoch [3/15], Train Loss: 0.0016, Val Loss: 0.0007
Epoch [4/15], Train Loss: 0.0032, Val Loss: 0.0015


In [ ]:
# Bestes Modell mit den gefundenen Hyperparametern trainieren
best_hp = study.best_trial.params
final_model = SimpleRNN(input_size=X_train.shape[2], hp=best_hp)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(final_model.parameters(), lr=best_hp['learning_rate'], weight_decay=best_hp['weight_decay'])

num_epochs = 50
train_loss_history = []
val_loss_history = []

for epoch in range(num_epochs):
    final_model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = final_model(X_batch)
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    train_loss_history.append(train_loss)

    # Validation Loss berechnen
    final_model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = final_model(X_batch)
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            val_loss += loss.item()
    val_loss /= len(val_loader)
    val_loss_history.append(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# **Trainingshistorie plotten**
plt.figure(figsize=(8, 5))
plt.plot(train_loss_history, label="Train Loss")
plt.plot(val_loss_history, label="Val Loss")
plt.xlabel("Epochen")
plt.ylabel("Loss")
plt.legend()
plt.title("Trainings- und Validierungsverlust")
plt.show()

torch.Size([16, 24, 17])